# Практика

In [ ]:
import pandas as pd
import numpy as np
import os

- 1) Загружаем данные. Имя файлов: data/output_20k-*.npz

   Информация о хитах по ключу hits
   
   Информация о энергии, координатам и импульсе события по ключу primaries

In [ ]:
files = os.listdir('data')
files.sort()

In [ ]:
data = np.load(f"data/{files[1]}", allow_pickle=True)
all_hits = data['hits']
all_primaries = data['primaries']

for file in files[2:10]:
    data = np.load(f"data/{file}", allow_pickle=True)
    hits = data['hits']
    primaries = data['primaries']

    all_hits = np.append(all_hits, hits)
    all_primaries = np.append(all_primaries, primaries)

- 2) Ознакомьтесь с размерностью массивов и их структурой (*полезно: .shape)

- 3) Создадим список со всеми энергиями событий. Энергия будет для нас целевой переменной

- 4) hits — это список сработавших ячеек в событии: [ [ix, iy, nHits, fHitTime_ns],  ... ]

#### Давайте визуалируем событие. Напишите функцию отрисовки, используя Plotly.
- У нас имеется два канала информации --- количество хитов на каждом модуле и время первого попадания (first hit time). Будет изображать их отдельно рядом друг с другом (plotly.subplots.make_subplots):
```python
    fig = make_subplots(
        rows=1, cols=2, column_widths=[0.5, 0.5],
        specs=[[{'type': 'scene'}, {'type': 'scene'}]]
    )
```
- Для визуализации зависимости значения от цвета можно использовать:
```python 
marker=dict(colorscale=your_colorscale)
```

- Для создания пустой сетки предлагаю использовать просто Scatter3d: 
```python 
    x_size = 16
    y_size = 16
    x_grid = np.arange(x_size)
    y_grid = np.arange(y_size)
    grid =  np.array(
        list(itertools.product(x_grid, y_grid))
    ) 
    
    grid_trace = go.Scatter3d(
                x=grid[:, 0],
                y=grid[:, 1],
                z=np.zeros(grid.shape[0]),
                mode='markers',
                showlegend=False,
                marker=dict(
                    symbol='square',
                    color='rgb(100, 100, 100)',
                    opacity=0.8
                )
            )
    
    fig.add_trace(grid_trace, row=1, col=1)
    fig.add_trace(grid_trace, row=1, col=2)
```

- Изобразите кол-во хитов и время первого попадания на базовой сетке, которую мы задали выше, используя различные цветовые гаммы 

- Изобразите истинную вершину события

- **Изобразите истинное направление (используйте px, py, pz)

In [ ]:
import itertools
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = 'plotly_dark'


def plot_event(hit, primary):
    pass

In [ ]:
n1 = np.random.randint(1000)
n2 = np.random.randint(1000)

plot_event(all_hits[n1], all_primaries[n1])
plot_event(all_hits[n2], all_primaries[n2])

- 5) Для реконструкции энергии будем использовать подход основанный на агрегированных признаках. То есть подсчитаем некоторые, обобщающие сигнал с оптических модулей, характеристики, как для канала с кол-во хитов, так и для временного канала.  

Такими характеристиками могут быть, например: 

- 1) Общее кол-во хитов на всех оптических модулях
- 2) Кол-во загоревшихся оптических модулей
- 3) Среднее значение времён первого попадания для загоревшихся опт. модулей
- 4) Дисперсия времён первого попадания для загоревшихся опт. модулей
- 5) ... Ваши идеи

**Подсчитайте их:**

**Обратите внимание, что бывают события, в которых не было ни одного хита!**

- 6) Сохраним наши признаки в формате pandas DataFrame. То есть в качестве колонок у нас будут признаки, а также добавим последнюю колонку с истинными энергиями: energies.

In [ ]:
features_df = pd.DataFrame()
features_df['nHits'] = nHits
features_df['nModules'] = nModules
features_df['x_cc'] = x_cc
features_df['y_cc'] = y_cc
features_df['r_cc'] = r_cc
features_df['fht_mean'] = fht_mean
features_df['fht_std'] = fht_std
features_df['edep'] = energies
features_df

- 7) Посмотрим на матрицу корреляций: 

In [ ]:
pio.templates.default = 'plotly_white'

corr_matrix = features_df.corr()

fig = go.Figure(data=go.Heatmap(
          x = corr_matrix.index,
          y = corr_matrix.columns,
          z = np.abs(corr_matrix.values),
          type = 'heatmap',
          xgap=0.25,
          ygap=0.25,
          colorscale = 'Inferno',
          name='Real')
)

fig.update_layout(
    width=600,
    height=600,
    xaxis_nticks=corr_matrix.index.shape[0],
    yaxis_nticks=corr_matrix.index.shape[0],
    font=dict(
        family="Times New Roman",
        size=15,
        color="Black"
    )
)

fig.show()

- 8) Посмотрим на зависимость Edep(nHits) с помощью go.Historgam2d, то есть 2D гистограммы.

Напишите функцию для отрисовки  2D гистограммы с помощью Plotly

In [ ]:
def corr_2d_vis(x, y, xaxis_title, yaxis_title):
    pass

In [ ]:
corr_2d_vis(features_df.nHits, features_df.edep, 'nHits', 'Deposited energy, Mev')

### Перейдем к обучении моделей:

- 9) Возьмите XGBRegressor из xgboost или постройне нейронную сеть, используя Keras. 
- Разделите данные на тренировочные и тестовые используя sklearn.model_selection.train_test_split
- Для нейронной сети не забудьте про нормализацию данных!
- Обучите базовую модель
- **Поподбирайте гиперпараметры, пооптимизируйте архитектуру

- 10) Используем нашу функции для 2D гистограмм для отрисовки результатов предсказания модели

In [ ]:
pred_df = pd.DataFrame([model.predict(X_val), y_val]).T
pred_df.columns = ['Edep_predict', 'Edep_true']

In [ ]:
corr_2d_vis(pred_df.Edep_predict, pred_df.Edep_true,
            'Reconstructed deposited energy, MeV', 'Deposited energy, MeV')

- 11) Теперь более внимательно посмотрим на ошибки

In [ ]:
dE = (pred_df.Edep_true - pred_df.Edep_predict) / pred_df.Edep_true * 100

In [ ]:
corr_2d_vis(pred_df.Edep_true, dE, 'Deposited energy, MeV',
            r"$\large dE = \frac{E_{\rm dep} - E_{\rm pred}}{E_{\rm dep}} \cdot 100, [\%]$",)

- 12) Выведите основные метрики для регрессии (MAPE, RMSE, MAE, R2, ...)